In [137]:
import pandas as pd 
import numpy as np
import re


### Cleaning Title

In [138]:
# load data ------
all_jobs = pd.read_csv("../data/cleaned/all_job.csv")
all_jobs = all_jobs.drop(["Unnamed: 0","job_id"], axis = 1)
all_jobs["title_category"] = all_jobs["title"]
last_col = all_jobs.pop(all_jobs.columns[-1])
all_jobs.insert(1, last_col.name, last_col)

#cleaning title ------
mask_ds = all_jobs['title_category'].str.contains('Data Scientist|data scientist|DATA SCIENTIST|Data Science')
all_jobs.loc[mask_ds, 'title_category'] = 'Data Scientist'

mask_da = all_jobs['title_category'].str.contains('Data Analyst|data analyst|DATA ANALYST|Data Analytics|Data analytics|Data analyst|Data & Analytics|Data & Technology Analyst')
all_jobs.loc[mask_da, 'title_category'] = 'Data Analyst'

mask_de = all_jobs['title_category'].str.contains('Data Engineer|data engineer|DATA ENGINEER|Data Engineering')
all_jobs.loc[mask_de, 'title_category'] = 'Data Engineer'

mask_se = all_jobs['title_category'].str.contains('Software Engineer|software engineer|SOFTWARE ENGINEER|SDE')
all_jobs.loc[mask_se, 'title_category'] = 'Software Engineer'

mask_bc = all_jobs['title_category'].str.contains('Blockchain|blockchain|Block Chain')
all_jobs.loc[mask_bc, 'title_category'] = 'Blockchain Engineer/Developer'

mask_ml = all_jobs['title_category'].str.contains('Machine Learning Engineer|MACHINE LEARNING|Machine Learning|machine learning engineering|ML')
all_jobs.loc[mask_ml, 'title_category'] = 'Machine Learning Engineer'

mask_dl = all_jobs['title_category'].str.contains('Deep Learning|Deep Reinforcement Learning|Neural Network|DEEP LEARNING ENGINEERS|Reinforcement Learning|neural networks|neural network')
all_jobs.loc[mask_dl, 'title_category'] = 'Deep Learning Engineer'

mask_bd = all_jobs['title_category'].str.contains('BIG DATA|big data|Big Data|Big data')
all_jobs.loc[mask_bd, 'title_category'] = 'Big Data Engineer/Architect'

mask_nlp = all_jobs['title_category'].str.contains('NLP|Natural Language Processing|Natural language processing')
all_jobs.loc[mask_nlp, 'title_category'] = 'Natural Language Processing Developer'

mask_rs = all_jobs['title_category'].str.contains('Research')
all_jobs.loc[mask_rs, 'title_category'] = 'Research Scientist'

mask_cloud = all_jobs['title_category'].str.contains('Cloud|cloud')
all_jobs.loc[mask_cloud, 'title_category'] = 'Cloud Engineer'

mask_ai = all_jobs['title_category'].str.contains('AI')
all_jobs.loc[mask_ai, 'title_category'] = 'AI Engineer'

categories = ['Data Analyst', 'Data Scientist', 'Data Engineer','Software Engineer','Blockchain Engineer/Developer','Machine Learning Engineer','Deep Learning Engineer','Big Data Engineer/Architect','Natural Language Processing Developer','Research Scientist','Cloud Engineer','AI Engineer']
mask_other = all_jobs['title_category'].isin(categories)
all_jobs.loc[~mask_other, 'title_category'] = 'Other Data Related Jobs'

all_jobs["title_category"].unique()


array(['Blockchain Engineer/Developer', 'Software Engineer',
       'Machine Learning Engineer', 'Data Analyst', 'Data Scientist',
       'Natural Language Processing Developer',
       'Big Data Engineer/Architect', 'Data Engineer', 'Cloud Engineer',
       'Research Scientist', 'Deep Learning Engineer',
       'Other Data Related Jobs', 'AI Engineer'], dtype=object)

## Language Tool counts

In [139]:
all_jobs['sql_count'] = all_jobs['description'].str.lower().apply(lambda x: len(re.findall(r'\bsql\b', x)))
all_jobs['python_count'] = all_jobs['description'].str.lower().apply(lambda x: len(re.findall(r'\bpython\b', x)))
all_jobs['R_count'] = all_jobs['description'].str.count(r'\bR\b')
all_jobs['java_count'] = all_jobs['description'].str.lower().apply(lambda x: len(re.findall(r'\bjava\b', x)))
all_jobs['linux_count'] = all_jobs['description'].str.lower().apply(lambda x: len(re.findall(r'\blinux\b', x)))
all_jobs['sas_count'] = all_jobs['description'].str.lower().apply(lambda x: len(re.findall(r'\bsas\b', x)))


tools = all_jobs[['title_category', 'sql_count','python_count','R_count','java_count','linux_count','sas_count']]
tools.to_csv("../data/cleaned/tools.csv")

# Degree

In [159]:
# mask_ai = all_jobs['title_category'].str.contains('AI')
# all_jobs.loc[mask_ai, 'title_category'] = 'AI Engineer'
all_jobs["phd"] = None
mask_phd = all_jobs['Qualification'].str.contains('phd|Ph.D|PhD|Doctorate')
all_jobs.loc[mask_phd, 'phd'] = 'PhD'
all_jobs["phd"]= all_jobs["phd"].fillna('')

all_jobs["master"] = None
mask_phd = all_jobs['Qualification'].str.contains('MS|Master\'s|MSc|M.S.|Graduate Degree|Master of Science')
all_jobs.loc[mask_phd, 'master'] = 'Master'
all_jobs["master"]= all_jobs["master"].fillna('')


all_jobs["bachelor"] = None
mask_phd = all_jobs['Qualification'].str.contains('Bachelor\’s.|BS')
all_jobs.loc[mask_phd, 'bachelor'] = 'Bachelor'
all_jobs["bachelor"] = all_jobs["bachelor"].fillna('')

all_jobs["hs"] = None
mask_phd = all_jobs['Qualification'].str.contains('GED|High School Diploma')
all_jobs.loc[mask_phd, 'hs'] = 'HighSchool'
all_jobs["hs"] = all_jobs["hs"].fillna('')



In [163]:
all_jobs["Degree Requirement"] = all_jobs["phd"]+ " " + all_jobs["master"] + " " +  all_jobs["bachelor"] + " "+ all_jobs["hs"]
all_jobs["Degree Requirement"] = all_jobs["Degree Requirement"].str.strip().replace(" ", ",")
all_jobs["Degree Requirement"] = all_jobs["Degree Requirement"].str.replace(r"\s+", ",")

/var/folders/_2/33mp53_x0hggvd5wvtm7xmhr0000gn/T/ipykernel_79102/27866767.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  all_jobs["Degree Requirement"] = all_jobs["Degree Requirement"].str.replace(r"\s+", ",")


In [166]:
all_jobs.to_csv("../data/cleaned/degree.csv")

# Get Locations

In [82]:
from geopy.geocoders import Nominatim


In [90]:
geolocator = Nominatim(user_agent='myapp')

def get_coordinates(location):
    try:
        geocode = geolocator.geocode(location)
        lat, lon = geocode.latitude, geocode.longitude
    except:
        lat, lon = None, None

    return lat, lon

all_jobs[['latitude', 'longitude']] = all_jobs.apply(lambda row: pd.Series(get_coordinates(row["location"])), axis=1)


In [91]:
all_jobs.to_csv('../data/cleaned/all_job_location.csv')

In [94]:
map = all_jobs[['location', 'latitude','longitude']]
map.to_csv("../data/cleaned/map.csv")